In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [ ]:
# Crear un directorio y mover el archivo `kaggle.json` allí.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Cambiar los permisos del archivo.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d  phucthaiv02/butterfly-image-classification

Dataset URL: https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification
License(s): CC0-1.0
 99% 224M/226M [00:14<00:00, 23.6MB/s]
100% 226M/226M [00:14<00:00, 16.7MB/s]


In [ ]:
!unzip butterfly-image-classification.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: train/Image_2348.jpg    
  inflating: train/Image_2349.jpg    
  inflating: train/Image_235.jpg     
  inflating: train/Image_2350.jpg    
  inflating: train/Image_2351.jpg    
  inflating: train/Image_2352.jpg    
  inflating: train/Image_2353.jpg    
  inflating: train/Image_2354.jpg    
  inflating: train/Image_2355.jpg    
  inflating: train/Image_2356.jpg    
  inflating: train/Image_2357.jpg    
  inflating: train/Image_2358.jpg    
  inflating: train/Image_2359.jpg    
  inflating: train/Image_236.jpg     
  inflating: train/Image_2360.jpg    
  inflating: train/Image_2361.jpg    
  inflating: train/Image_2362.jpg    
  inflating: train/Image_2363.jpg    
  inflating: train/Image_2364.jpg    
  inflating: train/Image_2365.jpg    
  inflating: train/Image_2366.jpg    
  inflating: train/Image_2367.jpg    
  inflating: train/Image_2368.jpg    
  inflating: train/Image_2369.jpg    
  inflating: train/Image_23

In [ ]:
#Importacion de librerias

import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch import nn
from torchvision import models, transforms
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import psutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from torchvision.transforms import Lambda
from tqdm import tqdm

from datetime import datetime


In [ ]:
# Configuración de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)



Using device: cuda


In [ ]:
# Configuraciones específicas del dataset


train_img_dir = './train'
train_csv_file = 'Training_set.csv'
train_df = pd.read_csv(train_csv_file)


train_df.head(3)



,filename,label
0,Image_1.jpg,SOUTHERN DOGFACE
1,Image_2.jpg,ADONIS
2,Image_3.jpg,BROWN SIPROETA


In [ ]:
label_list=list(train_df['label'].unique())
print(label_list)
print(len(label_list))
num_classes= print(len(label_list))

['SOUTHERN DOGFACE', 'ADONIS', 'BROWN SIPROETA', 'MONARCH', 'GREEN CELLED CATTLEHEART', 'CAIRNS BIRDWING', 'EASTERN DAPPLE WHITE', 'RED POSTMAN', 'MANGROVE SKIPPER', 'BLACK HAIRSTREAK', 'CABBAGE WHITE', 'RED ADMIRAL', 'PAINTED LADY', 'PAPER KITE', 'SOOTYWING', 'PINE WHITE', 'PEACOCK', 'CHECQUERED SKIPPER', 'JULIA', 'COMMON WOOD-NYMPH', 'BLUE MORPHO', 'CLOUDED SULPHUR', 'STRAITED QUEEN', 'ORANGE OAKLEAF', 'PURPLISH COPPER', 'ATALA', 'IPHICLUS SISTER', 'DANAID EGGFLY', 'LARGE MARBLE', 'PIPEVINE SWALLOW', 'BLUE SPOTTED CROW', 'RED CRACKER', 'QUESTION MARK', 'CRIMSON PATCH', 'BANDED PEACOCK', 'SCARCE SWALLOW', 'COPPER TAIL', 'GREAT JAY', 'INDRA SWALLOW', 'VICEROY', 'MALACHITE', 'APPOLLO', 'TWO BARRED FLASHER', 'MOURNING CLOAK', 'TROPICAL LEAFWING', 'POPINJAY', 'ORANGE TIP', 'GOLD BANDED', 'BECKERS WHITE', 'RED SPOTTED PURPLE', 'MILBERTS TORTOISESHELL', 'SILVER SPOT SKIPPER', 'AMERICAN SNOOT', 'AN 88', 'ULYSES', 'COMMON BANDED AWL', 'CRECENT', 'METALMARK', 'SLEEPY ORANGE', 'PURPLE HAIRSTREA

In [ ]:
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image

# Asegúrate de que label_list contenga todas las categorías únicas del DataFrame antes de este bucle
label_list = sorted(train_df['label'].unique())  # Ajusta 'category' al nombre de tu columna

prepared_images = []
labels = []  # Lista para almacenar los índices de las etiquetas

for image in tqdm(os.listdir(train_img_dir), desc="Preparing Images"):
    # Filtro para encontrar la categoría de la imagen actual
    filtre = train_df['filename'] == image
    if not filtre.any():
        continue  # Si no hay coincidencia, salta esta imagen
    kategori = train_df[filtre].values[0][1]  # Asegúrate de que este índice es correcto

    # Buscar el índice de la categoría en label_list
    try:
        kat = label_list.index(kategori)
        labels.append(kat)
    except ValueError:
        print(f"Category {kategori} not found in label_list.")
        continue  # O puedes asignar una categoría por defecto

    # Leer, redimensionar y normalizar la imagen
    image_path = os.path.join(train_img_dir, image)
    resim_oku = cv2.imread(image_path)
    if resim_oku is None:
        continue  # Si por alguna razón la imagen no se puede leer, salta
    resim180 = cv2.resize(resim_oku, (224, 224))
    resim = resim180 / 255.0  # Normalización
    flattened_image = resim.flatten()  # Aplanar la imagen
    prepared_images.append(flattened_image)

# Convertir listas a arrays de numpy para procesamiento posterior
prepared_images = np.array(prepared_images)
labels = np.array(labels)


Preparing Images:   0%|          | 0/6499 [00:00<?, ?it/s]

In [ ]:
print(len(prepared_images))
print(len(labels))

6499
6499


In [ ]:
x=np.array(prepared_images)
y=np.array(labels)

x.shape

(6499, 150528)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.70,random_state=35)

x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,test_size=0.5,random_state=35)



In [ ]:
 x_train.shape


(4549, 150528)

In [ ]:
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_test.shape, y_val.shape

((4549, 150528), (975, 150528), (975, 150528), (4549,), (975,), (975,))

### NAIVE VECTOR  - Regression Lineal

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Escalar las características
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

# Crear el modelo de regresión logística con un solucionador más rápido para grandes datasets
model = LogisticRegression(max_iter=5000, solver='saga', C=0.1)  # Ajusta estos parámetros según sea necesario

# Entrenar el modelo con los datos de entrenamiento escalados
model.fit(x_train_scaled, y_train)

# Predecir las etiquetas de los conjuntos de validación y prueba
y_val_pred = model.predict(x_val_scaled)
y_test_pred = model.predict(x_test_scaled)

# Calcular la precisión en los conjuntos de validación y prueba
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Crear el modelo KNN
model = KNeighborsClassifier(n_neighbors=3)  # Puedes ajustar el número de vecinos

# Entrenar el modelo con los datos de entrenamiento
model.fit(x_train, y_train)

# Predecir las etiquetas de los conjuntos de validación y prueba
y_val_pred = model.predict(x_val)
y_test_pred = model.predict(x_test)

# Calcular la precisión en los conjuntos de validación y prueba
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")


Validation Accuracy: 0.11
Test Accuracy: 0.12
